# Flaw Engine: Cloud Trainer (Gideon-gg101 Edition) 🚀

This notebook turns this Colab instance into a **Distributed Worker** for the Flaw Chess Engine.

### 🏁 Setup Options:
Run **ONLY ONE** of the setup options below.

## 1.1 Option A: Setup via GitHub (Recommended)
Use this if you have pushed your code to GitHub.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"

import os
# 1. Clean up everything
!rm -rf /content/flaw
print("Starting Clone...")

# 2. Clone from GitHub
!git clone {GITHUB_URL} /content/flaw

if os.path.exists('/content/flaw'):
    print("\n✅ GitHub Sync Successful!")
    %cd /content/flaw
else:
    print("\n❌ FAILED to clone. Ensure your repository is PUBLIC!")

## 1.2 Option B: Setup via Manual ZIP
Use this if GitHub is giving you path errors. Upload `flaw_cloud_bundle.zip` to the file explorer on the left.

In [ ]:
!rm -rf /content/flaw
!mkdir -p /content/flaw
!unzip -o /content/flaw_cloud_bundle.zip -d /content/flaw

import os
if os.path.exists('/content/flaw'):
    print("✅ Unzip Successful!")
    %cd /content/flaw
else:
    print("❌ FAILED to find ZIP. Did you upload 'flaw_cloud_bundle.zip'?")

## 2. Environment & Robust Build
Run this to compile the engine. It uses a Python deployment script for better reliability.

In [ ]:
!apt-get install -y cmake g++ > /dev/null
!pip install requests torch > /dev/null

import os
import shutil
import glob

if not os.path.exists('/content/flaw'):
    raise Exception("❌ Stop! Run Section 1 first.")

# 1. Build C++ Core
%cd /content/flaw
os.makedirs('core/build', exist_ok=True)
%cd core/build
!cmake .. > /dev/null
!make -j$(nproc) > /dev/null

# 2. Robust Library Deployment using Python
print("Deploying library files...")
lib_files = glob.glob("flaw_core*.so")
if not lib_files:
    raise Exception("❌ Build FAILED: No .so file found in core/build")

source_lib = os.path.abspath(lib_files[0])
targets = [
    '/content/flaw/flaw_core.so',
    '/content/flaw/ai/flaw_core.so'
]

for target in targets:
    if os.path.exists(target) or os.path.islink(target):
        os.remove(target)
    # Copying is safer than symlinking in some Cloud environments
    shutil.copy2(source_lib, target)
    print(f"✅ Deployed to: {target}")

%cd /content/flaw
print("\n✅ Environment Ready!")

## 3. Verify & Start Training
Replace the `MASTER_URL` with your local `ngrok` URL.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

import sys
import os

# 1. Setup Python Paths
ROOT_DIR = '/content/flaw'
if ROOT_DIR not in sys.path:
    sys.path.insert(0, ROOT_DIR)
os.environ['PYTHONPATH'] = f"{ROOT_DIR}:{os.environ.get('PYTHONPATH', '')}"

# 2. Verification Test
try:
    import flaw_core
    print("✅ C++ Core Loaded Successfully!")
    from ai.selfplay_worker import DistributedWorker
    print("✅ Worker Module Found!")
except Exception as e:
    print(f"❌ ERROR: Initialization failed: {e}")
    print("Diagnosis:")
    !ls -R /content/flaw/ai/flaw_core*
    raise

# 3. Launch Training
print("Starting Distributed Training...")
worker = DistributedWorker(MASTER_URL.rstrip('/'), depth=1)
worker.run()